In [ ]:
from shapely.geometry import LineString
import geopandas as gpd
import pandas as pd
import numpy as np

# Read the data
df_mine = pd.read_csv('./BM Vulcan/dgn_pit.str', 
                     names=['interior', 'x', 'y', 'z', 'none', 'type'])

# Clean data more efficiently - convert types during read with dtype parameter
df_mine = df_mine.drop(columns=['none']).dropna()
df_mine[['x', 'y', 'z']] = df_mine[['x', 'y', 'z']].astype(float)

# Pre-allocate lists for better performance
closed_line = []
levels = []

# Group by z-level to avoid repeated filtering
for level, group in df_mine.groupby('z'):
    # Convert to numpy arrays for faster operations
    coords = np.column_stack((group['y'].values, group['x'].values))
    
    # More efficient coordinate tracking
    coord_list = []
    for coord in coords:
        coord_tuple = tuple(coord)
        
        # Check if we've seen this coordinate before
        if coord_tuple in coord_list:
            # We found a duplicate, so close the line
            coord_list.append(coord_tuple)
            if len(coord_list) > 1:  # Make sure we have enough points for a LineString
                closed_line.append(LineString(coord_list))
                levels.append(level)
            
            # Reset for next line
            coord_list = []
        else:
            coord_list.append(coord_tuple)
    
    # Handle any remaining coordinates at the end
    if len(coord_list) > 1:
        closed_line.append(LineString(coord_list))
        levels.append(level)

# Create the GeoDataFrame all at once
gdf_closed_line = gpd.GeoDataFrame(
    data={'level': levels},
    geometry=closed_line, 
    crs="EPSG:32652"
)

# Display the result
gdf_closed_line.explore()